In [1]:
import re
import pandas as pd

In [2]:
raw_7_df = pd.read_excel(
    "./../raw_data/제_7_전국동시지방선거_개표/20180619-7지선-03-(시도의회의원)_읍면동별개표자료.xlsx",
    header=[0],
)

In [3]:
df = raw_7_df.copy()

In [4]:
# get splited_index
_df = df.dropna(subset=["선거종류"])
index_zip_list = list(zip(_df.index, _df.index[1:]))

In [5]:
# replacer
_replace_nan = lambda x: "" if type(x) != str else x
_replace_unnamed = lambda x: "후보자별 득표수" if x.startswith("Unnamed") else x
_replace_escape = lambda x: x.replace("_x000D_\n", "_")
_replace = lambda x: _replace_escape(_replace_unnamed(_replace_nan(x)))

_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace, x))

In [6]:
def replace_position(df, lib, con, etc):
    """
    lib : 진보당
    con : 보수당
    etc : 나머지당
    """
    df["lib"] = df.loc[:, lib].sum(axis=1)
    df["con"] = df.loc[:, con].sum(axis=1)
    df["etc"] = df.loc[:, etc].sum(axis=1)
    df.drop(columns=(lib + con + etc), inplace=True)
    return df


In [7]:
arg_list = list()
partial_df_list = list()
for index_zip in index_zip_list:
    # partial data
    partial_df = df.iloc[index_zip[0] : index_zip[1], :]

    # preproc columns
    multi_cols = [(col_a, col_b) for col_a, col_b in zip(partial_df.columns, partial_df.iloc[0, :])]
    flatten_cols = list(map(func, multi_cols))
    partial_df.columns = flatten_cols

    # drop duplicated data
    partial_df = partial_df.iloc[1:, 2:]
    partial_df
    # filter agg / etcs
    partial_df = partial_df[
        (partial_df["시도명"] != "합계")
        & (partial_df["구시군명"] != "합계")
        & ~(
            partial_df["읍면동명"].isin(
                ["합계", "거소투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
            )
        )
        & (partial_df["구분"] == "소계")
    ]
    partial_df = partial_df.loc[:, ~partial_df.columns.duplicated()]

    # renaming
    rename_dict = {
        "시도명": "loc_1",
        "구시군명": "loc_2",
        "읍면동명": "loc_3",
    }

    partial_df.rename(columns=rename_dict, inplace=True)
    partial_df.drop(columns=["구분", "후보자별 득표수"], inplace=True, errors="ignore")
    partial_df.columns = ["선거구명"] + list(partial_df.columns)[1:]
    
    #
    votes = list(filter(lambda x: x.startswith("후보자별 득표수"), partial_df.columns))
    partial_df.loc[:, votes] = partial_df.loc[:, votes].astype(int)
    lib = list(filter(lambda x: x.startswith("후보자별 득표수_더불어민주당"), partial_df.columns))
    con = list(filter(lambda x: x.startswith("후보자별 득표수_자유한국당"), partial_df.columns))
    etc = list(set(votes) - (set(lib) | set(con)))
    partial_df = replace_position(partial_df, lib, con, etc)

    # save
    argument = partial_df["loc_1"].iloc[0] + "_" + partial_df["선거구명"].iloc[0]
    arg_list.append(argument)
    partial_df_list.append(partial_df)

In [8]:
시도의회의원_7 = pd.concat(partial_df_list, axis=0)
시도의회의원_7.sample(10)

,선거구명,loc_1,loc_2,loc_3,선거인수,투표수,무효투표수,기권수,lib,con,etc
11582,영광군제2선거구,전라남도,영광군,염산면,3611.0,2515.0,180.0,1096.0,743,0.0,3927
9263,아산시제2선거구,충청남도,아산시,온양1동,8024.0,3977.0,77.0,4047.0,2204,1418,4178
13490,김해시제5선거구,경상남도,김해시,장유1동,38846.0,23396.0,468.0,15450.0,15956,6972,22928
2195,남구제1선거구,부산광역시,남구,대연제3동,26368.0,14708.0,432.0,11660.0,8467,5809,14276
4082,남구제3선거구,광주광역시,남구,효덕동,33490.0,19955.0,508.0,13535.0,13704,0.0,25190
2609,연제구제2선거구,부산광역시,연제구,연산제3동,7254.0,3999.0,82.0,3255.0,1742,1619,4473
2825,동구제2선거구,대구광역시,동구,신천4동,7572.0,3732.0,142.0,3840.0,0.0,2026,5154
13256,진주시제3선거구,경상남도,진주시,일반성면,2543.0,1871.0,82.0,672.0,654,1135,1789
6932,구리시제2선거구,경기도,구리시,수택3동,18521.0,11226.0,312.0,7295.0,7327,3587,10914
437,중랑구제3선거구,서울특별시,중랑구,묵제1동,28247.0,16596.0,400.0,11651.0,9884,3982,18526


In [9]:
시도의회의원_7.to_csv("./1st_preproc_data/시도의회의원_7.csv")

In [10]:
raw_8_df = pd.read_excel(
    "./../raw_data/제_8_전국동시지방선거_개표/시도의원선거.xlsx", header=[0, 1], thousands=","
)

In [11]:
df = raw_8_df.copy()

In [12]:
# preproc columns
_replace_unnamed = lambda x: "" if x.startswith("Unnamed") else x
_replace_candidate = lambda x: "" if re.match("^후보\d+", x) else x
_replace = lambda x: _replace_candidate(_replace_unnamed(x))

_join_strings = lambda x: "_".join(x).strip("_")

func = lambda x: _join_strings(map(_replace, x))

flatten_cols = list(map(func, df.columns))
df.columns = flatten_cols

In [13]:
# get splited_index
index_zip_list = list()
for arg in df["선거구명"].unique():
    _index = df[df["선거구명"] == arg].index
    index_zip_list.append((min(_index), max(_index)))

In [14]:
arg_list = list()
partial_df_list = list()

for index_zip in index_zip_list:
    #
    partial_df = df.iloc[index_zip[0] : index_zip[1], :]

    # preproc columns
    multi_cols = [(col_a, col_b) for col_a, col_b in zip(partial_df.columns, partial_df.iloc[0, :])]

    _replace_nan = lambda x: "" if type(x) != str else x
    _replace_unnamed = lambda x: "후보자별 득표수" if x.startswith("Unnamed") else x
    _replace_escape = lambda x: x.replace("\n", "_")
    _join_strings = lambda x: "_".join(x).strip("_")

    func = lambda x: _join_strings(
        map(_replace_escape, map(_replace_unnamed, map(_replace_nan, x)))
    )

    flatten_cols = list(map(func, multi_cols))
    partial_df.columns = flatten_cols

    #
    partial_df = partial_df.iloc[1:, :]

    #
    partial_df = partial_df[
        ~(
            partial_df["읍면동명"].isin(
                ["합계", "거소투표", "관외사전투표", "재외투표", "잘못 투입·구분된 투표지"]
            )
        )
        & (partial_df["구분"] == "소계")
    ]

    #
    _replace_default = lambda x: x.replace("_선거인수", "")
    partial_df.columns = [_replace_default(col) for col in partial_df.columns]
    partial_df.columns = ["loc_1", "loc_2", "선거구명", "loc_3"] + list(partial_df.columns)[4:]
    partial_df = partial_df.drop(columns=["구분"])
    partial_df = partial_df.loc[
        :, ["선거구명", "loc_1", "loc_2", "loc_3"] + list(partial_df.columns)[4:]
    ]
    partial_df = partial_df.loc[:, ~partial_df.columns.duplicated()]
    
    #
    votes = list(filter(lambda x: x.startswith("후보자별 득표수"), partial_df.columns))
    partial_df.loc[:, votes] = partial_df.loc[:, votes].astype(int)
    lib = list(filter(lambda x: x.startswith("후보자별 득표수_더불어민주당"), partial_df.columns))
    con = list(filter(lambda x: x.startswith("후보자별 득표수_국민의힘"), partial_df.columns))
    etc = list(set(votes) - (set(lib) | set(con)))
    partial_df = replace_position(partial_df, lib, con, etc)
    
    # save
    argument = partial_df["loc_1"].iloc[0] + "_" + partial_df["선거구명"].iloc[0]
    arg_list.append(argument)
    partial_df_list.append(partial_df)

In [15]:
시도의회의원_8 = pd.concat(partial_df_list, axis=0)
시도의회의원_8.head()

,선거구명,loc_1,loc_2,loc_3,선거인수,투표수,계,무효투표수,기권수,lib,con,etc
4,종로구제1선거구,서울특별시,종로구,청운효자동,9179.0,4885.0,4817.0,68.0,4294.0,2436,2381,0
7,종로구제1선거구,서울특별시,종로구,사직동,8057.0,4446.0,4368.0,78.0,3611.0,1761,2607,0
10,종로구제1선거구,서울특별시,종로구,삼청동,1962.0,1002.0,996.0,6.0,960.0,409,587,0
13,종로구제1선거구,서울특별시,종로구,부암동,7693.0,4135.0,4091.0,44.0,3558.0,1876,2215,0
16,종로구제1선거구,서울특별시,종로구,평창동,14730.0,7928.0,7834.0,94.0,6802.0,2975,4859,0


In [16]:
시도의회의원_8.to_csv("./1st_preproc_data/시도의회의원_8.csv")